In [5]:
from __future__ import print_function

import numpy as np
import tensorflow as tf

# Import MNIST data
import load_data as data

In [12]:
# n different points we want to cluster
# We have 59 points in all 3 categories * 2 people * 10 repetitions (maia_fall only had 9)
n = 59
# Tensor length of 80 since that's how many frames we decided on--easily modifiable
ts_len = 80

# Starting off with just one point so that it's easier to follow along with tutorials
# Will switch to more than one point later
# joint_index specifies which of the 36 joints we want, 0 is nose
# Find corresponding values at https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/output.md
joint_index = 0 

Xtr, Ytr = data.get_one_keypoint(n, joint_index)
Xte, Yte = data.get_one_keypoint(n, 2)
# Xtr, Ytr = mnist.train.next_batch(5000) #5000 for training (nn candidates)
# Xte, Yte = mnist.test.next_batch(200) #200 for testing

# tf Graph Input
xtr = tf.placeholder("float", [59, ts_len])
xte = tf.placeholder("float", [ts_len])

# Nearest Neighbor calculation using L1 Distance
# Calculate L1 Distance
distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)
# Prediction: Get min distance index (Nearest neighbor)
pred = tf.arg_min(distance, 0)

accuracy = 0.

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

mat/SliuTurn8.mat
mat/SliuTurn9.mat
mat/SliuJump9.mat
mat/SliuFall8.mat
mat/SliuFall9.mat
mat/SliuJump8.mat
mat/MaiaTurn10.mat
mat/MaiaTurn3.mat
mat/MaiaJump7.mat
mat/MaiaFall6.mat
mat/MaiaFall7.mat
mat/MaiaJump6.mat
mat/MaiaTurn2.mat
mat/SliuJump10.mat
mat/MaiaJump4.mat
mat/MaiaFall5.mat
mat/MaiaFall4.mat
mat/MaiaJump5.mat
mat/MaiaTurn1.mat
mat/MaiaTurn5.mat
mat/MaiaJump1.mat
mat/MaiaFall1.mat
mat/MaiaTurn4.mat
mat/MaiaTurn6.mat
mat/MaiaJump2.mat
mat/MaiaFall3.mat
mat/MaiaFall2.mat
mat/MaiaJump3.mat
mat/MaiaTurn7.mat
mat/MaiaTurn9.mat
mat/MaiaTurn8.mat
mat/MaiaJump10.mat
mat/MaiaJump8.mat
mat/MaiaFall9.mat
mat/MaiaFall8.mat
mat/MaiaJump9.mat
mat/SliuJump6.mat
mat/SliuTurn2.mat
mat/SliuFall7.mat
mat/SliuFall6.mat
mat/SliuTurn3.mat
mat/SliuJump7.mat
mat/SliuJump5.mat
mat/SliuTurn1.mat
mat/SliuFall4.mat
mat/SliuTurn10.mat
mat/SliuFall5.mat
mat/SliuJump4.mat
mat/SliuTurn4.mat
mat/SliuFall1.mat
mat/SliuFall10.mat
mat/SliuTurn5.mat
mat/SliuJump1.mat
mat/SliuJump3.mat
mat/SliuTurn7.mat
mat/S

In [13]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # loop over test data
    for i in range(len(Xte)):
        # Get nearest neighbor
        nn_index = sess.run(pred, feed_dict={xtr: Xtr, xte: Xte[i, :]})
        # Get nearest neighbor class label and compare it to its true label
        print("Test", i, "Prediction:", np.argmax(Ytr[nn_index]), \
            "True Class:", np.argmax(Yte[i]))
        # Calculate accuracy
        if np.argmax(Ytr[nn_index]) == np.argmax(Yte[i]):
            accuracy += 1./len(Xte)
    print("Done!")
    print("Accuracy:", accuracy)

Test 0 Prediction: 4 True Class: 3
Test 1 Prediction: 1 True Class: 3
Test 2 Prediction: 4 True Class: 4
Test 3 Prediction: 5 True Class: 5
Test 4 Prediction: 5 True Class: 5
Test 5 Prediction: 4 True Class: 4
Test 6 Prediction: 0 True Class: 0
Test 7 Prediction: 0 True Class: 0
Test 8 Prediction: 1 True Class: 1
Test 9 Prediction: 2 True Class: 2
Test 10 Prediction: 2 True Class: 2
Test 11 Prediction: 1 True Class: 1
Test 12 Prediction: 4 True Class: 0
Test 13 Prediction: 4 True Class: 4
Test 14 Prediction: 1 True Class: 1
Test 15 Prediction: 2 True Class: 2
Test 16 Prediction: 2 True Class: 2
Test 17 Prediction: 1 True Class: 1
Test 18 Prediction: 0 True Class: 0
Test 19 Prediction: 0 True Class: 0
Test 20 Prediction: 1 True Class: 1
Test 21 Prediction: 2 True Class: 2
Test 22 Prediction: 4 True Class: 0
Test 23 Prediction: 4 True Class: 0
Test 24 Prediction: 1 True Class: 1
Test 25 Prediction: 2 True Class: 2
Test 26 Prediction: 2 True Class: 2
Test 27 Prediction: 1 True Class: 1
Te